## git_checkout cipsrv_nhdplus_m

In [ ]:
import os,sys,git,tempfile,psycopg2,argparse;
from ipywidgets import IntProgress,HTML,VBox;
from IPython.display import display;
parser = argparse.ArgumentParser();
parser.add_argument("--override_branch" ,required=False);
args, _ = parser.parse_known_args();

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

gith = os.environ['GITHUB_REPO_URL'];
# override: "git@github.com:USEPA/CIPv2.git"
if args.override_branch is not None:
    brnh = args.override_branch;
else:
    brnh = os.environ['GITHUB_DEFAULT_BRANCH'];
# override: "mybranch"

dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv';
pasw = os.environ['POSTGRESQL_CIP_PASS'];

depf  = os.path.join('src','database','cipsrv_nhdplus_m','cipsrv_nhdplus_m_deploy.sql');

print("repo: " + gith);
print("branch: " + brnh);

In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pasw
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
except:
    raise Exception("database connection error");

print("database is ready");

In [ ]:
td = tempfile.TemporaryDirectory();

repo = git.Repo.clone_from(
     url     = gith
    ,branch  = brnh
    ,to_path = td.name 
    ,depth   = 1
);

print("checkout complete.");

In [ ]:
trg = os.path.join(td.name,depf);

if not os.path.exists(trg):
    raise Exception(trg + ' file not found.');
    
z = common.load_sqlfile(conn,trg,echo=True);


In [ ]:
conn.close();
td.cleanup();

print("logic load complete.");